# Vernon Inference

In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet("amalgamated.parquet")

In [3]:
vars = [
    "ETH-CAD_High",
    "ETH-CAD_Low",
    "BTC-CAD_High",
    "BTC-CAD_Low",
    "AdrBalNtv0.01Cnt",
    "AdrBalNtv0.1Cnt",
    "AdrBalNtv1Cnt",
    "AdrBalNtv10Cnt",
    "BlkSizeMeanByte",
    "CapRealUSD",
    "FeeByteMeanNtv",
    "FlowInExNtv",
    "FlowOutExNtv",
    "FlowTfrFromExCnt",
    "GasUsedTxMean",
    "NDF",
    "SplyAct1d",
    "SplyActPct1yr",
    "TxCnt",
    "VelCur1yr",
    'Global_Liquidity_Index'
]


df = df[vars]

In [4]:
# Assuming df is your DataFrame
#df = df.pct_change()*100

# The first row will be NaN because there's no previous data to subtract from the first entry
# If you wish to remove the NaN values, you can drop the first row
df.dropna(inplace=True)

df

,ETH-CAD_High,ETH-CAD_Low,BTC-CAD_High,BTC-CAD_Low,AdrBalNtv0.01Cnt,AdrBalNtv0.1Cnt,AdrBalNtv1Cnt,AdrBalNtv10Cnt,BlkSizeMeanByte,CapRealUSD,...,FlowInExNtv,FlowOutExNtv,FlowTfrFromExCnt,GasUsedTxMean,NDF,SplyAct1d,SplyActPct1yr,TxCnt,VelCur1yr,Global_Liquidity_Index
time,,,,,,,,,,,,,,,,,,,,,
2019-01-01,192.649857,180.575409,5246.870117,5057.234375,5725623,2553149,948660,240984,13665.922841,2.887769e+10,...,392978.197625,4.197261e+05,15602,73640.350123,0.635803,1.122109e+07,72.295246,448168,11.758691,10677.329758
2019-01-02,213.886246,191.795868,5388.129883,5191.100586,5729156,2554121,948649,241344,18126.132355,2.899081e+10,...,790231.396805,7.295684e+05,21191,60064.995705,0.636376,1.034321e+07,72.264375,589959,11.643999,11979.420951
2019-01-03,212.573837,198.451218,5362.917969,5159.898926,5733108,2555952,948798,240985,19342.790323,2.898172e+10,...,654169.139428,6.504445e+05,20336,56967.882419,0.636168,9.498194e+06,72.221087,596620,11.529535,11920.554140
2019-01-04,211.151260,199.472687,5172.040527,5087.920898,5732036,2553742,948728,240912,18872.124595,2.909478e+10,...,680241.364599,1.184203e+06,21142,61527.769639,0.636041,1.421223e+07,72.193628,549398,11.365511,11671.058247
2019-01-05,215.118729,206.441116,5223.182617,5131.535645,5730961,2549563,944626,240101,17351.082028,2.904683e+10,...,631193.304059,1.490288e+06,17419,63305.365839,0.637131,1.288664e+07,72.080216,511189,11.221790,10776.278107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-02,3088.287109,3013.004883,57877.652344,56241.656250,24732407,5195599,1729374,344511,149793.691422,1.667400e+11,...,224978.990905,3.353659e+05,103742,97339.905076,0.677616,1.248846e+07,35.971409,1108172,6.045875,122242.497035
2024-02-03,3107.525391,3067.862793,58432.644531,57157.976562,24733683,5185916,1728662,344395,153626.986371,1.665798e+11,...,88094.093613,1.106255e+05,93780,102296.542796,0.677607,1.176235e+07,35.937981,1054604,6.036129,121766.009277
2024-02-04,3136.216309,3090.657471,58429.691406,57797.507812,24723204,5169224,1728661,344281,153334.124877,1.664679e+11,...,155456.643029,1.523433e+05,89438,102710.415043,0.678024,9.964788e+06,35.919804,1049649,6.033078,122454.072998


## High Forecast

In [5]:
import torch
import numpy as np
import pandas as pd  # Ensure pandas is imported

from gluonts.torch.model.predictor import PyTorchPredictor
from gluonts.dataset.common import ListDataset
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator
from gluonts.transform.feature import MissingValueImputation

# Assuming df is your DataFrame with the data
target_column = 'ETH-CAD_High'  # Replace with your target column name

# Load the trained model
model_path = 'vernon_high.pth'
trained_predictor = torch.load(model_path)

# Ensure the DataFrame's index is a datetime index and set the frequency explicitly if needed
df.index = pd.to_datetime(df.index)
freq = "D"  # Adjust the frequency of your data as needed
df = df.asfreq(freq)

# Define the prediction length and context length
prediction_length = 1
context_length = 7

# Select dynamic features from the DataFrame, excluding the target column
past_dynamic_feature_columns = df.columns.drop(target_column)  # Excludes the target column
past_dynamic_features = df[past_dynamic_feature_columns].values.transpose()
past_dynamic_dims = [1] * len(past_dynamic_feature_columns)  # Adjust based on actual dynamic features

# Adjust the slicing for dynamic features for the test dataset
inference_past_dynamic_features_sliced = past_dynamic_features[:, -context_length:]

# Correct forecast start date
forecast_start_date = df.index[-1] + pd.Timedelta(days=1)

# Setup for inference using the last context_length days as input
inference_data = ListDataset([
    {
        "start": forecast_start_date,
        "target": df[target_column][-context_length:].values,
        "past_feat_dynamic_real": inference_past_dynamic_features_sliced
    }
], freq=freq)

# Initialize lists to store predictions for the 50th, 10th, and 90th percentiles
high_p50_predictions = []
high_p10_predictions = []
high_p90_predictions = []

# Perform inference and capture the desired quantiles
for forecast in trained_predictor.predict(inference_data):
    high_p50_predictions.append(forecast.quantile(0.5))  # Median
    high_p10_predictions.append(forecast.quantile(0.1))  # 10th Percentile
    high_p90_predictions.append(forecast.quantile(0.9))  # 90th Percentile

# Convert lists to numpy arrays
high_p50_predictions = np.array(high_p50_predictions).flatten()[:prediction_length]
high_p10_predictions = np.array(high_p10_predictions).flatten()[:prediction_length]
high_p90_predictions = np.array(high_p90_predictions).flatten()[:prediction_length]

# Output the sizes to confirm they match prediction_length and print the percentile predictions
print(f"10th percentile predictions: {high_p10_predictions}")
print(f"50th percentile (median) predictions: {high_p50_predictions}")
print(f"90th percentile predictions: {high_p90_predictions}")


10th percentile predictions: [3214.5051]
50th percentile (median) predictions: [3214.5347]
90th percentile predictions: [3214.5654]


## Low Forecast

In [6]:
import torch
import numpy as np
import pandas as pd  # Ensure pandas is imported

from gluonts.torch.model.predictor import PyTorchPredictor
from gluonts.dataset.common import ListDataset
from gluonts.torch.model.tft import TemporalFusionTransformerEstimator
from gluonts.transform.feature import MissingValueImputation

# Assuming df is your DataFrame with the data
target_column = 'ETH-CAD_Low'  # Replace with your target column name

# Load the trained model
model_path = 'vernon_low.pth'
trained_predictor = torch.load(model_path)

# Ensure the DataFrame's index is a datetime index and set the frequency explicitly if needed
df.index = pd.to_datetime(df.index)
freq = "D"  # Adjust the frequency of your data as needed
df = df.asfreq(freq)

# Define the prediction length and context length
prediction_length = 1
context_length = 7

# Select dynamic features from the DataFrame, excluding the target column
past_dynamic_feature_columns = df.columns.drop(target_column)  # Excludes the target column
past_dynamic_features = df[past_dynamic_feature_columns].values.transpose()
past_dynamic_dims = [1] * len(past_dynamic_feature_columns)  # Adjust based on actual dynamic features

# Adjust the slicing for dynamic features for the test dataset
inference_past_dynamic_features_sliced = past_dynamic_features[:, -context_length:]

# Correct forecast start date
forecast_start_date = df.index[-1] + pd.Timedelta(days=1)

# Setup for inference using the last context_length days as input
inference_data = ListDataset([
    {
        "start": forecast_start_date,
        "target": df[target_column][-context_length:].values,
        "past_feat_dynamic_real": inference_past_dynamic_features_sliced
    }
], freq=freq)

# Initialize lists to store predictions for the 50th, 10th, and 90th percentiles
low_p50_predictions = []
low_p10_predictions = []
low_p90_predictions = []

# Perform inference and capture the desired quantiles
for forecast in trained_predictor.predict(inference_data):
    low_p50_predictions.append(forecast.quantile(0.5))  # Median
    low_p10_predictions.append(forecast.quantile(0.1))  # 10th Percentile
    low_p90_predictions.append(forecast.quantile(0.9))  # 90th Percentile

# Convert lists to numpy arrays
low_p50_predictions = np.array(low_p50_predictions).flatten()[:prediction_length]
low_p10_predictions = np.array(low_p10_predictions).flatten()[:prediction_length]
low_p90_predictions = np.array(low_p90_predictions).flatten()[:prediction_length]

# Output the sizes to confirm they match prediction_length and print the percentile predictions
print(f"10th percentile predictions: {low_p10_predictions}")
print(f"50th percentile (median) predictions: {low_p50_predictions}")
print(f"90th percentile predictions: {low_p90_predictions}")


10th percentile predictions: [3122.636]
50th percentile (median) predictions: [3122.7139]
90th percentile predictions: [3122.7961]


## Predictions

In [7]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Adjusting the start and end times
frequency = "1d"
start_time = (datetime.now(pytz.timezone('US/Pacific')) - timedelta(days=1)).strftime('%Y-%m-%d')  # 5 days ago from the current date
end_time = (datetime.now(pytz.timezone('US/Pacific')) + timedelta(days=1)).strftime('%Y-%m-%d')  # Current date in US/Pacific

# Retrieve historical data and store it in a dictionary
ETHc_df = yf.download("ETHH.TO", start=start_time, end=end_time, interval=frequency)[['High', 'Low']]
ETHc_df

[*********************100%%**********************]  1 of 1 completed


,High,Low
Date,,
2024-02-05,9.43,9.27
2024-02-06,9.70,9.47


In [8]:
import pandas as pd
import numpy as np

# Assuming the variables for 10th and 90th percentile predictions are defined and populated
# Ensure all predictions are properly shaped
low_p10_predictions = np.array(low_p10_predictions).flatten()
low_p90_predictions = np.array(low_p90_predictions).flatten()
high_p10_predictions = np.array(high_p10_predictions).flatten()
high_p90_predictions = np.array(high_p90_predictions).flatten()

# Generate forecast dates starting from the day after the last day in the dataset
forecast_dates = pd.date_range(start=forecast_start_date, periods=prediction_length, freq=freq)

# Check if the length of predictions matches the expected prediction_length for all
assert len(low_p50_predictions) == prediction_length, "Mismatch in low_p50_predictions length"
assert len(high_p50_predictions) == prediction_length, "Mismatch in high_p50_predictions length"
assert len(low_p10_predictions) == prediction_length, "Mismatch in low_p10_predictions length"
assert len(low_p90_predictions) == prediction_length, "Mismatch in low_p90_predictions length"
assert len(high_p10_predictions) == prediction_length, "Mismatch in high_p10_predictions length"
assert len(high_p90_predictions) == prediction_length, "Mismatch in high_p90_predictions length"

# Create a DataFrame with forecast dates and predictions
forecast_df = pd.DataFrame({
    'Date': forecast_dates,
    'Low_P10_Prediction': low_p10_predictions,
    'Low_P50_Prediction': low_p50_predictions,
    'Low_P90_Prediction': low_p90_predictions,
    'High_P10_Prediction': high_p10_predictions,
    'High_P50_Prediction': high_p50_predictions,
    'High_P90_Prediction': high_p90_predictions
})

# Optionally, set the date as the index of the DataFrame
forecast_df.set_index('Date', inplace=True)

# Display the DataFrame
forecast_df / 332.4296  # Assuming the division by 332.4296 is specific to your use case


,Low_P10_Prediction,Low_P50_Prediction,Low_P90_Prediction,High_P10_Prediction,High_P50_Prediction,High_P90_Prediction
Date,,,,,,
2024-02-07,9.393375,9.39361,9.393857,9.669732,9.669821,9.669913
